In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
#for installing all the libraries
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install transformers
!pip install simpletransformers
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━

In [4]:
#for checking gpu details if present any on machine
!nvidia-smi

Sat Aug  3 15:00:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
#for loading evaluate function
!pip install evaluate
!pip install tensorflow-text


In [13]:
#for importing necessary all the libraries
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch
import nltk
import evaluate

In [14]:
# to load .jsonl dataset files in the code and returning dataframe of important columns for generation task.(mainly two columns, content and their respective spoilers)
def load_dataset(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:

        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            paragraphs = ' '.join(example['targetParagraphs'])
            spoiler = example['spoiler'][0]
            data.append({'text': post_text + ' - ' + title + paragraphs, 'spoiler': spoiler})
    return pd.DataFrame(data)
train_data = load_dataset('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.jsonl')
validation_data = load_dataset('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/val.jsonl')

In [15]:
# converting both dataframes to numpy to further apply preprocessing on it.
train_data = np.array(train_data)
validation_data = np.array(validation_data)

In [16]:
# For removing all the extra special characters from the data
characters = ['!','"','#','$','%','&','(',')','*','+','/',':',';','<','=','>','@','^','`','|','~','\t','[',']','{','}','\\','.','-']
for i in range(len(train_data)):
    for j in characters:
        train_data[i][0] = train_data[i][0].replace(j,"")
        train_data[i][1] = train_data[i][1].replace(j,"")

for i in range(len(validation_data)):
    for j in characters:
        validation_data[i][0] = validation_data[i][0].replace(j,"")
        validation_data[i][1] = validation_data[i][1].replace(j,"")


print(len(train_data))
print(len(validation_data))

train_data = pd.DataFrame(train_data)
validation_data = pd.DataFrame(validation_data)

3200
400


In [17]:
# Converting dataframes and storing them as .csv file
train_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv', index=False)
validation_data.to_csv('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/val.csv', index=False)

In [14]:
!cd /content/drive/MyDrive/Colab Notebooks/Pegasus_data/data



drive  sample_data


In [18]:
# loading dataset in the form which is acceptable to pretrained model
from datasets import load_dataset

dataset_file = r'/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv'
dataset_file_1 = r'/content/drive/MyDrive/Colab Notebooks/Pegasus_data/data/train.csv'
train_data = load_dataset('csv', data_files=dataset_file)
val_data = load_dataset('csv', data_files=dataset_file_1)

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
print(train_data['train'][1])

{'0': "NASA sets date for full recovery of ozone hole  Hole In Ozone Layer Expected To Make Full Recovery By 2070 NASA2070 is shaping up to be a great year for Mother Earth That's when NASA scientists are predicting the hole in the ozone layer might finally make a full recovery Researchers announced their conclusion, in addition to other findings, in a presentation Wednesday during the annual American Geophysical Union meeting in San Francisco The team of scientists specifically looked at the chemical composition of the ozone hole, which has shifted in both size and depth since the passing of the Montreal Protocol in 1987 The agreement banned its 197 signatory countries from using chemicals, like chlorofluorocarbons CFCs, that break down into chlorine in the upper atmosphere and harm the ozone layer They found that, while levels of chlorine in the atmosphere have indeed decreased as a result of the protocol, it's too soon to tie them to a healthier ozone layer Ozone holes with smaller 

In [21]:
# loading pretrained model pegasus-large on gpu of the server along with it's tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_ckpt = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [22]:
# Creating embeddings of the data
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['0'] , max_length = 256, truncation = True , padding= "max_length")

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['1'], max_length = 50, truncation = True , padding= "max_length")

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train = train_data.map(convert_examples_to_features, batched = True)
val = val_data.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [31]:
print(val['train']['input_ids'])

Output hidden; open in https://colab.research.google.com to view.

In [22]:
#Intializing data collector
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [23]:
#Defining training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

trainer_args = Seq2SeqTrainingArguments(
    output_dir=r'/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus', num_train_epochs=10, warmup_steps=0,
    per_device_train_batch_size=2, per_device_eval_batch_size=2,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='epoch', save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# Passing trainer arguments
trainer = Seq2SeqTrainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train['train'],
                  eval_dataset=val['train'])

In [25]:
# for starting training of the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,6.957700,6.401539
2,1.871800,0.748453
3,0.499400,0.351218
4,0.288900,0.292613
5,0.285500,0.260503
6,0.340700,0.240669
7,0.261600,0.229219
8,0.216200,0.223157
9,0.240800,0.217332
10,0.274200,0.216174


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=1000, training_loss=1.509690771818161, metrics={'train_runtime': 9556.0942, 'train_samples_per_second': 3.349, 'train_steps_per_second': 0.105, 'total_flos': 2.3115715313664e+16, 'train_loss': 1.509690771818161, 'epoch': 10.0})

In [26]:
# For saving model after training
model_pegasus.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Pretrained")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [27]:
#For saving it's tokenizer after saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens")

('/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Pegasus_data/saved_model/Pegasus_Tokens/tokenizer.json')

In [ ]:
import shutil
shutil.move("/content/location where ypu want to store your model", "/content/drive/MyDrive/Pegasus_data/saved_model")

'/content/drive/MyDrive/Pegasus_data/saved_model/location where ypu want to store your model'